In [ ]:
#!pip install docarray

# LangChain Expression Language (LCEL)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)


## Simple Chain

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser


In [ ]:
chain.invoke({"topic": "bears"})

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [ ]:
#from langchain_community.embeddings import BedrockEmbeddings
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.invoke("where did harrison work?")

In [ ]:
retriever.invoke("what do bears like to eat")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | output_parser

In [ ]:
chain.invoke({"question": "where did harrison work?"})

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "where did harrison work?"})

## Bind

and OpenAI Functions

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "input_schema": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
).bind(tools=functions)


In [ ]:
runnable = prompt | llm

In [ ]:
runnable.invoke({"input": "what is the weather in sf"})

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "input_schema": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "input_schema": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]


In [ ]:
llm = llm.bind(tools=functions)

In [ ]:
runnable = prompt | llm

In [ ]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

## Fallbacks

In [ ]:
from langchain_aws import BedrockLLM
import json

model_kwargs_llama = {"temperature": 0}

In [ ]:
simple_model = BedrockLLM(
    model_id="meta.llama3-8b-instruct-v1:0",
    model_kwargs=model_kwargs_llama
)
simple_chain = simple_model | json.loads

In [ ]:
challenge = "write three poems. output in json format with keys: 'title', 'author', and 'first_line'"

In [ ]:
simple_model.invoke(challenge)

**Note**: The next line is expected to fail.

In [ ]:
simple_chain.invoke(challenge)

In [ ]:
better_model = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    model_kwargs=dict(temperature=0)
)
chain = better_model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

## Interface

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

In [ ]:
for t in chain.stream({"topic": "bears"}):
    print(t)

In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response